In [1]:
import csv
import glob
from iso639 import Lang
import pycountry
from langcodes import *
import os
import pandas as pd

def addToDict(word, reps,dictionary):
    if word in dictionary:
        dictionary[word]+=reps
    else:
        dictionary[word]=reps


csvsToAnalyze=glob.glob("./../../../data/non_filtered/corpora/pud/*.csv")
for csvfile in csvsToAnalyze:
    with open(csvfile, newline='',encoding="utf-8") as csvfileOpened:
        reader = csv.reader(csvfileOpened, delimiter=',', quotechar='"')
        next(reader)
        names=csvfile.split("\\")[1].split("_")[0]
        isoLan=names
        language=pycountry.languages.get(alpha_3=Lang(isoLan).pt3)
        isoLanMod=language.name
        isoLanIdentifier=names
        isoLan=Lang(isoLanIdentifier).pt1
        characterFreq={}
        totLetters=0
        if "pinyin" in csvfile:
            characterFreq={}
            for row in reader:
                for letter in row[3]:
                    totLetters+=int(row[1])
                    addToDict(letter,int(row[1]),characterFreq)
        if "romaji" in csvfile:
            for row in reader:
                for letter in row[3]:
                    totLetters+=int(row[1])
                    addToDict(letter,int(row[1]),characterFreq)
        else:
            for row in reader:
                for letter in row[0]:
                    totLetters+=int(row[1])
                    addToDict(letter,int(row[1]),characterFreq)
        language=pycountry.languages.get(alpha_3=Lang(isoLan).pt3)
        toAdd=""
        if "strokes" in csvfile:
            toAdd+="-strokes"
        if "pinyin" in csvfile:
            toAdd+="-pinyin"
        if "romaji" in csvfile:
            toAdd+= "-romaji"

        with open("./../../../data/non_filtered/alphabets/pud/"+Lang(isoLan).pt3+toAdd+"-character.csv", 'w', encoding='UTF8', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["character", "frequency", "frequencyRel"])
            for w in sorted(characterFreq, key=characterFreq.get, reverse=True):
                    writer.writerow([w,characterFreq[w], characterFreq[w]/totLetters])
